## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2021/seasontype/2

In [1]:
SET_WEEK = 13
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_13


In [2]:
from scrapers.scrape_2021 import get_2021_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2021 data

In [4]:
%%time

X = get_2021_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game', 'offense_passing_AVG', 'offense_passing_YDS/G', 'offense_passing_RTG', 
             'offense_receiving_AVG', 'offense_rushing_YDS/G', 'defense_points_per_game', 'defense_passing_AVG', 
             'defense_receiving_AVG', 'defense_rushing_YDS/G', 'defense_passing_SYL_per_game', 
             'offense_downs_Third Downs_ATT_per_game', 'defense_downs_Third Downs_ATT_per_game', 
             'defense_downs_Fourth Downs_ATT_per_game', 'defense_pass_to_rush_ratio', 
             'defense_downs_First Downs_penalty_ratio', 'offense_passing_TD_per_game', 'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

Wall time: 18.1 s


,Team,games_played,season,offense_points_per_game,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,offense_rushing_YDS/G,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_Fourth Downs_ATT_per_game,defense_pass_to_rush_ratio,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,offense_pass_TD_to_INT
0,Dallas Cowboys,12,2021,29.4,7.7,287.3,101.4,11.3,129.0,22.3,...,12.4,111.1,15.916667,13.083333,12.333333,1.166667,1.471380,0.121339,2.083333,2.777778
1,Tampa Bay Buccaneers,12,2021,31.4,7.4,311.1,103.9,10.8,92.6,22.5,...,9.8,84.3,16.666667,12.333333,12.833333,1.333333,1.919679,0.120000,2.833333,3.400000


In [5]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [6]:
X['games_played'].unique().tolist()

[12, 13]

In [8]:
# X[X['games_played'] == 8]

In [9]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 4

predictions.sort_values('pred_mean', ascending = False, inplace = True)
predictions.head(3)

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
4,Los Angeles Rams,2021,week_13,0.285693,0.182141,0.312164,0.306866,0.271716
9,Arizona Cardinals,2021,week_13,0.248226,0.212166,0.293918,0.274384,0.257173
1,Tampa Bay Buccaneers,2021,week_13,0.243647,0.190015,0.272759,0.285410,0.247958


In [10]:
predictions.to_csv('weekly_runs/' + week_name + '.csv', index = False)

### Check weekly run

In [11]:
predictions

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
4,Los Angeles Rams,2021,week_13,0.285693,0.182141,0.312164,0.306866,0.271716
9,Arizona Cardinals,2021,week_13,0.248226,0.212166,0.293918,0.274384,0.257173
1,Tampa Bay Buccaneers,2021,week_13,0.243647,0.190015,0.272759,0.285410,0.247958
8,Buffalo Bills,2021,week_13,0.232852,0.170518,0.187534,0.252737,0.210910
17,New England Patriots,2021,week_13,0.160766,0.149496,0.125581,0.227272,0.165779
0,Dallas Cowboys,2021,week_13,0.197098,0.121180,0.142112,0.191658,0.163012
7,Las Vegas Raiders,2021,week_13,0.157015,0.103311,0.131311,0.205377,0.149254
11,San Francisco 49ers,2021,week_13,0.133512,0.079116,0.141441,0.228657,0.145682
13,Cincinnati Bengals,2021,week_13,0.148496,0.079995,0.124429,0.181568,0.133622
5,Minnesota Vikings,2021,week_13,0.118790,0.058078,0.097336,0.185542,0.114937
